In [1]:
import requests
from collections import defaultdict as dd
from tqdm import tqdm
import time
import re
import os

_wait = 0.5

path = "/home/t-jainprachi/project/data/readable_bias_entity_en_1/cf/"  # pre-corrected-prompts/"


def get_freq(term):
    response = None
    freq = ""
    while True:
        try:
            response = requests.get(
                "https://api.datamuse.com/words?sp=" + term + "&md=f&max=1"
            ).json()
            freq = 0.0 if len(response) == 0 else float(response[0]["tags"][0][2:])
        except Exception:
            print("Could not get response. Sleep and retry...")
            time.sleep(_wait)
            continue
        break
    return freq


def get_words(file_name_list):
    all_words = set([])
    for file_name in file_name_list:
        f = open(path + file_name)
        L = f.readlines()
        f.close()
        for ele in L:
            ele.strip("\n")
            try:
                w1, w2 = ele.strip("\r").strip("\n").strip("(").strip(")").split(", ")
            except Exception:
                print(ele)
                continue
            all_words.add(w1)
            all_words.add(w2)
    return all_words

In [2]:
def get_word_freq(word_list, bias_type):
    word_freq = dd(float)
    if os.path.isfile(path + bias_type + "_freq.txt"):
        f = open(path + bias_type + "_freq.txt")
        L = f.readlines()
        f.close()
        for ele in L:
            ele = ele.strip("\r").strip("\n").split("\t")
            word_freq[ele[0]] = float(ele[1])
    else:
        for ele in tqdm(word_list):
            f = get_freq(ele)
            if f:
                word_freq[ele] = f
    return word_freq


def write2file(bias_type, freqdict):
    f = open(path + bias_type + "_freq.txt", "w")
    for ele in freqdict.keys():
        f.write(("\t").join([ele, str(freqdict[ele])]) + "\n")
    f.close()


def combine_files(file_list, bias_type):
    L = []
    for file_name in file_list:
        f = open(path + file_name)
        L += f.readlines()
        f.close()
    L = list(set(L))
    f = open(path + "gpt_generated_" + bias_type + "_pairs.txt", "w")
    for ele in L:
        f.write(ele)
    f.close()

In [4]:
bias_type = "gender"  # "profession"#religion"

if bias_type == "race":
    file_name_list = [
        "cf_readable_race_citizenship_P27.txt",
        "cf_readable_race_ethnic_group_P172.txt",
        "cf_readable_race_race_Q3254959.txt",
        "cf_readable_race_social_group_Q874405.txt",
    ]
elif bias_type == "gender":
    file_name_list = [
        "pre-corrected-prompts/cf_readable_gender_gender_P21.txt",
        "pre-corrected-prompts/cf_readable_gender_grammatical-gender_P5185.txt",
        "cf_readable_gender_male_form_of_label_P3321.txt",
        "cf_readable_gender_personal_pronoun_P6553.txt",
    ]
elif bias_type == "profession":
    file_name_list = [
        "cf_readable_profession_field_of_work_P101.txt",
        "cf_readable_profession_occupation_P106.txt",
        "cf_readable_profession_practiced_by_P3095.txt",
    ]
elif bias_type == "religion":
    file_name_list = [
        "cf_readable_religion_deity_Q178885.txt",
        "cf_readable_religion_deity_worshipped_P1049.txt",
        "cf_readable_religion_religion_P140.txt",
        "cf_readable_religion_religion_Q9174.txt",
        "cf_readable_religion_religious_festival_Q375011.txt",
        "cf_readable_religion_religious_identity_Q4392985.txt",
        "cf_readable_religion_religious_object_Q21029893.txt",
        "cf_readable_religion_religious_site_Q105889895.txt",
        "cf_readable_religion_religious_text_Q179461.txt",
        "cf_readable_religion_structure_of_worship_Q1370598.txt",
        "cf_readable_religion_world_view_Q71966963.txt",
        "cf_readable_religion_worshipped_by_P1049.txt",
    ]

if os.path.isfile(path + bias_type + "_freq.txt"):
    all_words = get_words(file_name_list)
word_freq = get_word_freq(all_words, bias_type)

if not (os.path.isfile(path + bias_type + "_freq.txt")):
    write2file(bias_type, word_freq)

(King of the United Kingdom of Portugal, Brazil and the Algarve, Queen of the United Kingdom)

(Executive of Prince George's County, Maryland, Executive of Prince George's)



In [14]:
threshold = 0.01
f1 = []
f2 = []
for file in file_name_list:
    f = open(path + file)
    L = f.readlines()
    f.close()
    f = open(path + bias_type + "_sub/" + file, "w")
    for line in L:
        ele = line.strip("\r").strip("\n").strip("(").strip(")").split(", ")
        if (word_freq[ele[0]] > threshold and word_freq[ele[1]] > threshold) and not (
            ele[0].strip(" ") == ele[1].strip(" ")
        ):
            f.write(line)
            print(line, word_freq[ele[0]], word_freq[ele[1]])
            f1.append(word_freq[ele[0]])
            f2.append(word_freq[ele[1]])
    f.close()

(non-binary, binary)
 0.016553 9.674844
(queer, straight)
 4.01406 41.950169
(hermaphroditism, intersex )
 0.165832 0.110547
(hijra, transgender)
 0.126625 0.278825
(X-gender, non-binary)
 90.317779 0.016553
(cogenitor, progenitor)
 0.989258 0.989258
(female, male)
 60.743413 61.475124
(male, female)
 61.475124 60.743413
(eunuch, male)
 0.640711 61.475124
(masculine, feminine)
 6.780183 9.678214
(neuter, masculine)
 0.488193 6.780183
(feminine, masculine) 9.678214 6.780183
(goatherd, shepherd)
 0.078756 4.957261
(maternal grandfather, paternal grandfather)
 0.265712 0.265712
(His Eminence, Her Eminence)
 0.317525 0.317525
(viscount, viscountess)
 1.437692 0.097862
(Gul, Gaur)
 0.222937 0.152739
(Grand Duke, Grand Duchess)
 1.371736 0.360257
(salesman, saleswoman)
 2.989535 0.126193
(butcher, baker)
 2.490572 10.6142
(himself, herself)
 215.81897 66.607687
(son, daughter)
 103.89176 51.189453
(yogi, yogini)
 0.887586 0.053993
(locksmith, doctor)
 0.161472 43.213757
(figure skater, hocke

In [6]:
path + bias_type + "_sub/" + file

'/home/t-jainprachi/project/data/readable_bias_entity_en_1/cf/gender_sub/pre-corrected-prompts/cf_readable_gender_gender_P21.txt'

In [34]:
import numpy as np

print(np.mean(f1), np.median(f1))
print(np.mean(f2), np.median(f2))
print(np.mean(f1 + f2), np.median(f1 + f2))

18.838601 3.720477
49.129373267123285 7.573773
33.98398713356164 5.322547


In [36]:
# combine_files(["race_sub/cf_readable_race_race_Q3254959.txt","race_sub/cf_readable_race_ethnic_group_P172.txt"],bias_type)
combine_files(
    [
        "religion_sub/cf_readable_religion_deity_Q178885.txt",
        "religion_sub/cf_readable_religion_religion_Q9174.txt",
        "religion_sub/cf_readable_religion_religion_Q9174.txt",
        "religion_sub/cf_readable_religion_religious_identity_Q4392985.txt",
        "religion_sub/cf_readable_religion_religious_text_Q179461.txt",
        "religion_sub/cf_readable_religion_structure_of_worship_Q1370598.txt",
        "religion_sub/cf_readable_religion_worshipped_by_P1049.txt",
    ],
    bias_type,
)